In [1]:
# Imports
import simpy
import random
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pygame
import time
import os.path
random.seed(76)

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
rand = random.randint(90,95)/100
PARROQUIA = 'EL VECINO'
TIPO_EDUCACION = 'REGULAR'
SOSTENIMIENTO = 'Fiscal'
MODALIDAD = 'Prescencial'
AULAS = 36*2
TOTAL_DOCENTES = round(104*0.9)
TOTAL_ESTUDIANTES = round(2001*rand)
ESTUDIANTES_CONTAGIADOS = round(TOTAL_ESTUDIANTES*0.02)
ESTUDIANTES_POR_AULA = round(TOTAL_ESTUDIANTES/AULAS)
HORAS_DIA = 6
HORA_RECESO = 0.5
TOTAL_DIAS = HORAS_DIA * 120
AULAS_CERRADAS = 0

DATA = []

In [3]:
class Jornada(object):
    def __init__(self, environment, aulas, docentes, estudiantes, parroquia, tipo_educacion, sostenimiento, modalidad):
        self.env = environment
        self.aulas = aulas
        self.docentes = simpy.Resource(environment, docentes)
        self.estudiantes = estudiantes
        self.parroquia = parroquia
        self.tipo_educacion = tipo_educacion
        self.sostenimiento = sostenimiento
        self.modalidad = modalidad
    def curso(self):
        try:
            if(env.now < 180):
                print('Nuevo día iniciado a las: ', env.now)
                yield self.env.timeout(HORAS_DIA)
                self.contagio()
                print('Termina el día a las: ', env.now)
                DATA.append([(env.now)/6, self.docentes.capacity, self.estudiantes - ESTUDIANTES_CONTAGIADOS, ESTUDIANTES_CONTAGIADOS, self.aulas, AULAS_CERRADAS])
            else:
                print('Nuevo día iniciado a las: ', env.now)
                yield self.env.timeout(HORAS_DIA - HORA_RECESO)
                yield self.env.timeout(HORA_RECESO)
                self.contagio()
                print('Termina el día a las: ', env.now)
                self.prueba()
                DATA.append([(env.now)/6, self.docentes.capacity, self.estudiantes - ESTUDIANTES_CONTAGIADOS, ESTUDIANTES_CONTAGIADOS, self.aulas, AULAS_CERRADAS])
        except simpy.Interrupt:
            print("Interrupción")
            
    def prueba(self):
        global AULAS_CERRADAS
        global ESTUDIANTES_CONTAGIADOS 
        estudiantes_prueba = round(ESTUDIANTES_POR_AULA * 0.1)
        print('Se hace la prueba a ', estudiantes_prueba)
        foco_contagio = random.randint(1,100)
        if(foco_contagio < 3):
            print('######## ESTUDIANTE CONTAGIADO, se cierra un curso ###########')
            self.aulas -= 1
            self.docentes = simpy.Resource(self.env, self.docentes.capacity-1)
            self.estudiantes -= round(ESTUDIANTES_POR_AULA*0.8)
            ESTUDIANTES_CONTAGIADOS += round(ESTUDIANTES_POR_AULA*0.2)
            AULAS_CERRADAS += 1
                
    def contagio(self):
        global ESTUDIANTES_CONTAGIADOS 
        ESTUDIANTES_CONTAGIADOS += round((self.estudiantes - ESTUDIANTES_CONTAGIADOS)*0.02)

In [4]:
def simulacion(env, jornada):
    with jornada.docentes.request() as profesor:
        yield profesor
        yield env.process(jornada.curso())

def ejecutar_simulacion(env, aulas, docentes, estudiantes, parroquia, tipo_educacion, sostenimiento, modalidad):
    while True:
        yield env.process(simulacion(env, Jornada(env, aulas, docentes, estudiantes, parroquia, tipo_educacion, sostenimiento, modalidad)))

In [5]:
print("Iniciando retorno a clases")
env = simpy.Environment()
env.process(ejecutar_simulacion(env, AULAS, TOTAL_DOCENTES, TOTAL_ESTUDIANTES, PARROQUIA, TIPO_EDUCACION, SOSTENIMIENTO, MODALIDAD))
env.run(until=TOTAL_DIAS)
print("")
print("")
print("--- RESULTADOS ---")
print("Total de cursos: ", AULAS)
print("Cursos Cerrados: ", AULAS_CERRADAS)
print("Docentes Disponibles: ", TOTAL_DOCENTES - AULAS_CERRADAS)

Iniciando retorno a clases
Nuevo día iniciado a las:  0
Termina el día a las:  6
Nuevo día iniciado a las:  6
Termina el día a las:  12
Nuevo día iniciado a las:  12
Termina el día a las:  18
Nuevo día iniciado a las:  18
Termina el día a las:  24
Nuevo día iniciado a las:  24
Termina el día a las:  30
Nuevo día iniciado a las:  30
Termina el día a las:  36
Nuevo día iniciado a las:  36
Termina el día a las:  42
Nuevo día iniciado a las:  42
Termina el día a las:  48
Nuevo día iniciado a las:  48
Termina el día a las:  54
Nuevo día iniciado a las:  54
Termina el día a las:  60
Nuevo día iniciado a las:  60
Termina el día a las:  66
Nuevo día iniciado a las:  66
Termina el día a las:  72
Nuevo día iniciado a las:  72
Termina el día a las:  78
Nuevo día iniciado a las:  78
Termina el día a las:  84
Nuevo día iniciado a las:  84
Termina el día a las:  90
Nuevo día iniciado a las:  90
Termina el día a las:  96
Nuevo día iniciado a las:  96
Termina el día a las:  102
Nuevo día iniciado a la

In [6]:
df = pd.DataFrame(DATA, columns=['Día', 'Docentes', 'Estudiantes Sanos', 'Estudiantes Contagados', 'Aulas', 'Aulas Cerradas'])
df

,Día,Docentes,Estudiantes Sanos,Estudiantes Contagados,Aulas,Aulas Cerradas
0,1.0,94,1768,73,72,0
1,2.0,94,1733,108,72,0
2,3.0,94,1698,143,72,0
3,4.0,94,1664,177,72,0
4,5.0,94,1631,210,72,0
...,...,...,...,...,...,...
114,115.0,94,169,1672,72,3
115,116.0,94,166,1675,72,3
116,117.0,94,163,1678,72,3
117,118.0,94,160,1681,72,3


In [7]:
count = 0
for i in range(df.index.stop):
    fig, ax = plt.subplots(figsize=(10,5))
    ax.barh(df.columns, df.loc[i])
    ax.text(0.95, 0.85, 'Día '+str(i+1), transform=ax.transAxes, size=26, ha='right')
    ax.set_xticks(np.arange(0, 1810, 200))
    plt.savefig('plots/{}.png'.format(count))
    count += 1
    plt.close()

In [8]:
pygame.init()
width = 1000
height = 600
window = pygame.display.set_mode((width, height))
pygame.display.set_caption('Gráfico')
clock = pygame.time.Clock()

In [ ]:
def graph(count):
    img = pygame.image.load('plots/{}.png'.format(count))
    window.fill((255,255,255))
    window.blit(img, (100,10))
    
running = True
count = 0
while(running):
#    for event in pygame.event.get():
#        if event.type == pygame.QUIT:
#            running = False
#            pygame.quit()
#            window.fill((0,0,0))
#    if event.type == pygame.KEYDOWN:
#        if event.key == pygame.K_SPACE:
    for i in range(445):
#        for event in pygame.event.get():
#            if event.type == pygame.QUIT():
#                running = False
#                pygame.quit()
        graph(count)
        pygame.display.update()
        time.sleep(0.2)
        count += 1
        if count >=119:
            count = 0
        clock.tick(70)
    time.sleep(2)
    running = False
pygame.quit()